In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import gc
from matplotlib import pyplot as plt
import latent.loop as loop
import latent.session_functional as session
import latent.utils.plot_utils as plot_utils 
import latent.utils.stats_utils as stat
import latent.utils.data_utils as prd
import latent.ml.dl_functional as dl
from latent.ml.lda import train_lda
import os
import copy as cp
from matplotlib import gridspec
import matplotlib as mpl
set_gpu()

plt.rcParams['figure.dpi'] = 300
%matplotlib qt
mpl.rc('font', family='serif') 
mpl.rc('font', serif='Palatino Linotype') 
mpl.rc('font', size=12)


Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'TR'
with open('traindata/train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,feat,feat_sq = pickle.load(f)

In [ ]:
# Loop through training
train_dict = {'sub_type':sub_type,'n_train':'fullallmix4', 'load':False, 'train_scale':5, 'epochs': 30, 'batch_size' : 128, 'sparsity':True,'dt':'all','feat_type':'feat','noise':True,'gens':50, 'mod_dt':'emgscalelim','train_grp':2,'latent_dim':4}
train_sess = session.Session(**train_dict)
dt_all = ['all']
mod_all = ['emgscalelim']

for i in range(5):
    train_out = [None] * np.max(params[:,0])

    # loop through subjects
    for sub_i in range(1,np.max(params[:,0])+1):
        print('sub: ' + str(sub_i))
        train_out[sub_i-1] = train_sess.loop_cv(raw,params,sub=sub_i,mod=['all'])
        gc.collect()
    
    with open('AB_aug_' + str(i) + '.p', 'wb') as f:
        pickle.dump([train_out],f)
    
    gc.collect()


In [ ]:
time_all = [None] * 5
aug_all = [None] * 5
for i in range(5):
    with open('AB_timings_' + str(i)+ '.p', 'rb') as f:
        time_all[i] = pickle.load(f)
    with open('AB_aug_' + str(i)+ '.p', 'rb') as f:
        aug_all[i] = pickle.load(f)

In [ ]:
sum_sae = np.full((5*np.max(params[:,0]),),np.nan)
sum_cnn = np.full((5*np.max(params[:,0]),),np.nan)
sum_lda = np.full((5*np.max(params[:,0]),),np.nan)
sum_aug = np.full((5*np.max(params[:,0]),),np.nan)

it = 0
for i in range(5):
    for sub_i in range(1,np.max(params[:,0])+1):
        sum_sae[it] = time_all[i][0][sub_i-1]['sae_time'] + time_all[i][0][sub_i-1]['sae_a_time'] + aug_all[i][0][sub_i-1]['noise_time']
        sum_cnn[it] = time_all[i][0][sub_i-1]['cnn_time'] + time_all[i][0][sub_i-1]['cnn_a_time'] + aug_all[i][0][sub_i-1]['noise_time']
        sum_lda[it] = time_all[i][0][sub_i-1]['lda_time']
        sum_aug[it] = time_all[i][0][sub_i-1]['aug lda time'] + aug_all[i][0][sub_i-1]['noise_time']
        it += 1

mean_sae = np.nanmean(sum_sae)
mean_cnn = np.nanmean(sum_cnn)
mean_lda = np.nanmean(sum_lda)
mean_aug = np.nanmean(sum_aug)

std_sae = np.nanstd(sum_sae)
std_cnn = np.nanstd(sum_cnn)
std_lda = np.nanstd(sum_lda)
std_aug = np.nanstd(sum_aug)


In [ ]:
# reduce dimensions of inputs
sub = 5
train_dict = {'sub_type':sub_type,'n_train':'fullallmix4', 'load':False, 'train_scale':5, 'epochs': 30, 'batch_size' : 128, 'sparsity':True,'dt':'all','feat_type':'feat','noise':True,'gens':50, 'mod_dt':'emgscalelim','train_grp':2,'latent_dim': 4}
train_sess = session.Session(**train_dict)


test_dict = {'sub_type':sub_type,'dt':'all', 'mod_dt':'emgscalelim','sparsity':True, 'load':True, 'batch_size':128, 'latent_dim':4, 'epochs':30,'train_scale':5, 'n_train':'fullallmix4', 'n_test':'posrealmixeven','feat_type':'feat', 'noise':True,'train_grp':2,'test_dt': 'noisescalelim'}
test_sess = session.Session(**test_dict)
ntype = 'posrealmixeven'
addon = False
traintest = 'trainnoise'

if not addon:
    x_clean_lda = np.array([]).reshape(0,6)
    x_clean_noise = np.array([]).reshape(0,6)
    x_clean_sae = np.array([]).reshape(0,6)#test_sess.latent_dim)
    x_clean_cnn = np.array([]).reshape(0,6)#test_sess.latent_dim)
    y_clean = np.array([]).reshape(0,1)
    y_noise_clean = np.array([]).reshape(0,1)
    max_i = 2

    if traintest == 'testnoise':
        x_noisy_lda = np.array([]).reshape(0,6)
        x_noisy_noise = np.array([]).reshape(0,6)
        x_noisy_sae = np.array([]).reshape(0,6)#test_sess.latent_dim)
        x_noisy_cnn = np.array([]).reshape(0,6)#test_sess.latent_dim)
        y_noisy = np.array([]).reshape(0,1)
        max_i = 5

if ntype == 'flat':
    test_max = 2
else:
    test_max = 6

for i in range(1,max_i):
    if traintest == 'testnoise':
        test_sess.n_test = 'part' + ntype + str(i) + '4'
        red_out = test_sess.reduce_latent(raw, params, sub,traintest=traintest)
    else:
        red_out = train_sess.reduce_latent(raw, params, sub,traintest=traintest)
    for key,val in red_out.items():
        exec(key + '=val')
    x_clean_lda = np.vstack([x_test_lda_red[:clean_size_lda,:], x_clean_lda])
    x_clean_noise = np.vstack([x_test_noise_red[:clean_size,:], x_clean_noise])
    x_clean_sae = np.vstack([x_test_sae_red[:clean_size,:], x_clean_sae])
    x_clean_cnn = np.vstack([x_test_cnn_red[:clean_size,:], x_clean_cnn])
    y_clean = np.vstack([y_test[:clean_size_lda,:], y_clean])
    y_noise_clean = np.vstack([y_test_noise[:clean_size,:], y_noise_clean])
    print(clean_size)
    print(clean_size_lda)

    x_noisy_lda = np.vstack([x_test_lda_red[clean_size_lda:,:], x_noisy_lda])
    x_noisy_noise = np.vstack([x_test_noise_red[clean_size:,:], x_noisy_noise])
    x_noisy_sae = np.vstack([x_test_sae_red[clean_size:,:], x_noisy_sae])
    x_noisy_cnn = np.vstack([x_test_cnn_red[clean_size:,:], x_noisy_cnn])
    y_noisy = np.vstack([y_test[clean_size_lda:,:], y_noisy])

x_all_lda = np.vstack([x_clean_lda,x_noisy_lda])
x_all_noise = np.vstack([x_clean_noise,x_noisy_noise])
x_all_sae = np.vstack([x_clean_sae,x_noisy_sae])
x_all_cnn = np.vstack([x_clean_cnn,x_noisy_cnn])
y_all = np.vstack([y_clean, y_noisy])

lda_lims = tuple(np.vstack((np.min(x_all_lda[:,:3],axis=0),np.max(x_all_lda[:,:3],axis=0))).T)
noise_lims = tuple(np.vstack((np.min(x_all_noise[:,:3],axis=0),np.max(x_all_noise[:,:3],axis=0))).T)
sae_lims = tuple(np.vstack((np.min(x_all_sae[:,:3],axis=0),np.max(x_all_sae[:,:3],axis=0))).T)
cnn_lims = tuple(np.vstack((np.min(x_all_cnn[:,:3],axis=0),np.max(x_all_cnn[:,:3],axis=0))).T)


In [ ]:
# Plot reduced dimensions
dim = 3
train_downsamp = clean_size//clean_size_lda
std_lim = False
mult = 2

In [ ]:
fig = plt.figure(figsize=(8,6))
plot_utils.plot_latent_rep(x_clean_lda, y_clean,fig,loc=1,lims=lda_lims,downsamp=train_downsamp,dim=dim, lim_min=lda_min, lim_max=lda_max,std_lim=std_lim,mult = mult)
plot_utils.plot_latent_rep(x_clean_noise, y_noise_clean,fig,loc=2,lims=noise_lims,downsamp=train_downsamp,dim=dim, lim_min=noise_min, lim_max=noise_max,std_lim=std_lim,mult = mult)
plot_utils.plot_latent_rep(x_clean_sae, y_noise_clean,fig,loc=3,lims=sae_lims,dim=dim,downsamp=train_downsamp, lim_min=sae_min, lim_max=sae_max,std_lim=std_lim,mult = mult)
plot_utils.plot_latent_rep(x_clean_cnn, y_noise_clean,fig,loc=4,lims=cnn_lims,dim=dim,downsamp=train_downsamp, lim_min=cnn_min, lim_max=cnn_max,std_lim=std_lim,mult = mult)
# plt.subplots_adjust(wspace=0, hspace=0)
fig.subplots_adjust(left=0.1, right=.9, bottom=0.1, top=.9)

In [ ]:
downsamp = 2
fig = plt.figure(figsize=(8,6))
lda_min, lda_max = plot_utils.plot_latent_rep(x_noisy_lda, y_noisy,fig,loc=1,lims=lda_lims,downsamp=downsamp,dim=dim,std_lim=std_lim,mult=mult)
noise_min, noise_max = plot_utils.plot_latent_rep(x_noisy_noise, y_noisy,fig,loc=2,lims=noise_lims,downsamp=downsamp,dim=dim,std_lim=std_lim,mult=mult)
sae_min, sae_max = plot_utils.plot_latent_rep(x_noisy_sae, y_noisy,fig,loc=3,lims=sae_lims,downsamp=downsamp,dim=dim,std_lim=std_lim,mult=mult)
cnn_min, cnn_max = plot_utils.plot_latent_rep(x_noisy_cnn, y_noisy,fig,loc=4,lims=cnn_lims,downsamp=downsamp,dim=dim,std_lim=std_lim,mult=mult)
# plt.subplots_adjust(wspace=0, hspace=0)
fig.subplots_adjust(left=0.1, right=.9, bottom=0.1, top=.9)

In [ ]:
fig = plt.figure()
plot_utils.plot_latent_rep(x_all_lda, y_all,fig,loc=1,lims=lda_lims,downsamp=downsamp,dim=dim)
plot_utils.plot_latent_rep(x_all_noise, y_all,fig,loc=2,lims=noise_lims,downsamp=downsamp,dim=dim)
plot_utils.plot_latent_rep(x_all_sae, y_all,fig,loc=3,lims=sae_lims,downsamp=downsamp,dim=dim)
plot_utils.plot_latent_rep(x_all_cnn, y_all,fig,loc=4,lims=cnn_lims,downsamp=downsamp,dim=dim)
plt.tight_layout()

In [ ]:
with open('real_noise/all_real_noise.p', 'rb') as f:
    real_noise_temp, _ = pickle.load(f)
    
plot_utils.plot_all_noise(real_noise_temp)


In [ ]:
all_noise = ['breaknm','break','contact','move']

# plot accuracy vs # noisy electrodes
fig = plt.figure(figsize=(12, 4)) 
gs = gridspec.GridSpec(8, 3)

for i in range(4):
    ntype = 'posreal' + all_noise[i]
    with open(sub_type + '_' + ntype + '1.p', 'rb') as f:
        xtestnoise,xtest,y_out = pickle.load(f)
    plot_utils.plot_noisy(xtestnoise,xtest,y_out,iter=i, gs=gs)

fig.subplots_adjust(left=0.1, right=.9, bottom=0.2, top=.9)

In [6]:
# sub_type = 'AB'
test_dict = {'sub_type':sub_type,'dt':'manual', 'mod_dt':'emgscalelim','sparsity':True, 'load':True, 'batch_size':128, 'latent_dim':4, 'epochs':30,'train_scale':5, 'n_train':'fullallmix4', 'n_test':'partgauss4','feat_type':'feat', 'noise':True,'train_grp':2,'test_dt':'noisescalelim'}
test_sess = session.Session(**test_dict) 

dt_all = ['all']
mod_all = ['emgscalelim']
test_all = ['noisescalelim']
load = True
if load:
    posi_max = 2
else:
    posi_max = 5

for dt in dt_all:
    test_sess.dt = dt
    for j in range(1):
        test_sess.mod_dt = mod_all[j]
        test_sess.test_dt = test_all[j]
        for posi in range(1,posi_max):
            ntype = 'posrealmixeven'
            
            if not load:
                ntype += str(posi)

            if ntype[:3] == 'pos' and not load:
                i_start = 4
                i_end = 5
                acc_all = np.full([np.max(params[:,0]), 3, 4, 15],np.nan)
                acc_clean = np.full([np.max(params[:,0]), 3, 4, 15],np.nan)
                acc_noise = np.full([np.max(params[:,0]), 3, 4, 15],np.nan)
            elif ntype == 'flat' or 'mix' in ntype or 'real' in ntype:
                i_start = 1
                i_end = 5
                acc_all = np.full([np.max(params[:,0]), 4, 1, 15],np.nan)
                acc_clean = np.full([np.max(params[:,0]), 4, 1, 15],np.nan)
                acc_noise = np.full([np.max(params[:,0]), 4, 1, 15],np.nan)
            else:
                i_start = 1
                i_end = 5
                acc_all = np.full([np.max(params[:,0]), 4, 5, 15],np.nan)
                acc_clean = np.full([np.max(params[:,0]), 4, 5, 15],np.nan)
                acc_noise = np.full([np.max(params[:,0]), 4, 5, 15],np.nan)

            for i in range(i_start,i_end):
                if load and 'pos' in ntype:
                    test_sess.n_test = 'part' + ntype + str(i) + '4'
                else:
                    test_sess.n_test = 'part' + ntype + str(i)    

                if load:
                    for sub in range(1,np.max(params[:,0])+1):
                        foldername = test_sess.create_foldername(ftype='results')
                        # foldername += '/rev2_noreg'
                        foldername += '/temp'
                        filename = test_sess.create_filename(foldername, 1, sub)
                        print(filename + '_' + test_sess.n_test)
                        if os.path.isfile(filename + '_' + test_sess.n_test + '_subresults.p'):
                            with open(filename + '_' + test_sess.n_test + '_subresults.p', 'rb') as f:
                                temp_all, temp_noise, temp_clean = pickle.load(f)
                            acc_all[sub-1,i-i_start,:,:], acc_clean[sub-1,i-i_start,:,:], acc_noise[sub-1,i-i_start,:,:] = np.squeeze(temp_all),np.squeeze(temp_clean),np.squeeze(temp_noise)
                else:
                    # test_out, x_test_noise, x_test_clean, y_test_clean = test_sess.loop_test(raw, params)
                    test_out = test_sess.loop_test(raw, params)
                    # print(test_out)
                    for key,val in test_out.items():
                        exec(key + '=val')

            ave_pos_noise= np.nanmean(acc_noise,axis=0)
            ave_pos_clean = np.nanmean(acc_clean,axis=0)
            ave_noise= np.nanmean(acc_noise,axis=2)
            ave_clean = np.nanmean(acc_clean,axis=2)
            ave_gauss_noise = np.nanmean(ave_noise,axis=0)
            ave_gauss_clean = np.nanmean(ave_clean,axis=0)

            if load:
                plot_utils.plot_electrode_results(ave_noise,ave_clean,test_sess.n_train,ntype,test_sess.sub_type)

            if 'gauss' in ntype:
                ave_gauss= cp.deepcopy(ave_noise)
            elif 'flat' in ntype:
                ave_flat = cp.deepcopy(ave_noise)
            elif '60hz' in ntype:
                ave_60hz = cp.deepcopy(ave_noise)
            elif 'mix' in ntype:
                ave_mix = cp.deepcopy(ave_noise)

results_2_all_emgscalelim_noisescalelim/temp/TR1_feat_dim_4_ep_30_bat_128_fullallmix4_5_lr_10_sparse_partposrealmixeven14
results_2_all_emgscalelim_noisescalelim/temp/TR2_feat_dim_4_ep_30_bat_128_fullallmix4_5_lr_10_sparse_partposrealmixeven14
results_2_all_emgscalelim_noisescalelim/temp/TR3_feat_dim_4_ep_30_bat_128_fullallmix4_5_lr_10_sparse_partposrealmixeven14
results_2_all_emgscalelim_noisescalelim/temp/TR4_feat_dim_4_ep_30_bat_128_fullallmix4_5_lr_10_sparse_partposrealmixeven14
results_2_all_emgscalelim_noisescalelim/temp/TR5_feat_dim_4_ep_30_bat_128_fullallmix4_5_lr_10_sparse_partposrealmixeven14
results_2_all_emgscalelim_noisescalelim/temp/TR6_feat_dim_4_ep_30_bat_128_fullallmix4_5_lr_10_sparse_partposrealmixeven14
results_2_all_emgscalelim_noisescalelim/temp/TR7_feat_dim_4_ep_30_bat_128_fullallmix4_5_lr_10_sparse_partposrealmixeven14
results_2_all_emgscalelim_noisescalelim/temp/TR1_feat_dim_4_ep_30_bat_128_fullallmix4_5_lr_10_sparse_partposrealmixeven24
results_2_all_emgscaleli

c:\ProgramData\Anaconda3\envs\latent_env\lib\site-packages\ipykernel_launcher.py:70: RuntimeWarning: Mean of empty slice
c:\ProgramData\Anaconda3\envs\latent_env\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning: Mean of empty slice


In [5]:
ave_noise2 = cp.deepcopy(ave_noise)
ave_clean2 = cp.deepcopy(ave_clean)
acc_clean2 = cp.deepcopy(acc_clean)
acc_noise2 = cp.deepcopy(acc_noise)
# ave_mix2 = cp.deepcopy(ave_mix)

In [7]:
ave_noise2[:,:,4] = ave_noise[:,:,6]
ave_clean2[:,:,4] = ave_clean[:,:,6]
ave_noise2[:,:,5] = ave_noise[:,:,7]
ave_clean2[:,:,5] = ave_clean[:,:,7]
acc_clean2[...,4] = acc_clean[...,6]
acc_noise2[...,4] = acc_noise[...,6]
acc_clean2[...,5] = acc_clean[...,7]
acc_noise2[...,5] = acc_noise[...,7]
# ave_mix2[:,:,4] = ave_mix[:,:,6]
# ave_mix2[:,:,5] = ave_mix[:,:,7]
# ave_noise2[:,:,11] = ave_noise[:,:,11]
# ave_clean2[:,:,11] = ave_clean[:,:,11]
# acc_clean2[...,11] = acc_clean[...,11]
# acc_noise2[...,11] = acc_noise[...,11]

In [ ]:
plot_utils.plot_nn_results(ave_noise,ave_clean,ave_noise2, ave_clean2, test_sess.n_train,ntype,test_sess.sub_type)

In [ ]:
plot_utils.plot_aug_results(ave_noise,ave_clean,ave_noise2, ave_clean2, test_sess.n_train,ntype,test_sess.sub_type)

In [8]:
mm = 1/25.4 
w = 210
h = w/2
# plot accuracy vs # noisy electrodes
fig = plt.figure(figsize=(w*mm, h*mm)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 2.75])
plot_utils.plot_electrode_results_aug(ave_noise2,ave_clean2,test_sess.n_train,ntype,test_sess.sub_type,gs=gs[0])
diff_mix,c ,c1 = plot_utils.plot_summary_aug(ave_clean2,ave_noise2,gs=gs[1])
fig.set_tight_layout(True)

In [ ]:
from matplotlib import gridspec
px = 1/plt.rcParams['figure.dpi']  # pixel in inches
w = 1200
h = w/2.125
# plot accuracy vs # noisy electrodes
fig = plt.figure(figsize=(w*px, h*px)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 2])
acc, std = plot_utils.plot_electrode_results(ave_noise,ave_clean,test_sess.n_train,ntype,test_sess.sub_type,gs=gs[0])
plot_utils.plot_summary(ave_clean,ave_mix,gs=gs[1])
fig.set_tight_layout(True)

mm = 1/25.4 
w = 200
h = w/2.125
# plot accuracy vs # noisy electrodes
fig = plt.figure(figsize=(w*mm, h*mm)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 2])
plot_utils.plot_electrode_results(ave_noise,ave_clean,test_sess.n_train,ntype,test_sess.sub_type,gs=gs[0])
plot_utils.plot_summary(ave_clean,ave_mix,gs=gs[1])
fig.set_tight_layout(True)

In [ ]:
x = np.linspace(0,0.2,200)
temp = []
gauss = []
fig, ax = plt.subplots(11,1)
for a in range(1,6):
    temp.append(a*np.sin(2*np.pi*60*x))
    gauss.append(np.random.normal(0,a,len(x)))
    ax[a-1].plot(x*1000,temp[a-1],'b',label='A = ' + str(a))
    ax[a+4].plot(x*1000,gauss[a-1],label='A = ' + str(a))

ax[-1].plot(x*1000,np.zeros(x.shape))
# ax.legend()
for i in range(11):
    if i > 0: 
        ax[i].set_yticks([])
        ax[i].set_yticklabels([])
    if i < 10:
        ax[i].set_xticks([])
        ax[i].set_xticklabels([])
    ax[i].set_ylim([-5,5])
    ax[i].set_xlim([0,200])
ax[-1].set_xlabel('Time (ms)')
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
from latent.ml.lda import predict, train_lda
from latent.ml.dl_subclass import MLP, CNN
import time
import timeit

test_dict = {'sub_type':sub_type,'dt':'all', 'mod_dt':'emgscalelim','sparsity':True, 'load':True, 'batch_size':128, 'latent_dim':4, 'epochs':30,'train_scale':5, 'n_train':'fullallmix4', 'n_test':'partposrealmixeven14','feat_type':'feat', 'noise':True,'train_grp':2,'test_dt':'noisescalelim'}
test_sess = session.Session(**test_dict) 

sae_time = [None] * np.max(params[:,0])
cnn_time = [None] * np.max(params[:,0])
lda_time = [None] * np.max(params[:,0])
aug_time = [None] * np.max(params[:,0])

# set number of models to test
mod_tot = 15
# set testing noise type
noise_type = test_sess.n_test[4:-1]
print(noise_type)

# load real_noise
if 'real' in noise_type:
    with open('real_noise/all_real_noise.p', 'rb') as f:
        real_noise_temp, _ = pickle.load(f)

cv_tot = 1
test_sess.max_cv = cv_tot + 1

filename = 0

# Set folder
foldername = test_sess.create_foldername()
resultsfolder = test_sess.create_foldername(ftype='results')

num_feats = 10

# loop through subjects
sub = 1
# index based on training group and subject
ind = (params[:,0] == sub) & (params[:,3] == test_sess.train_grp)

# Check if training data exists
if np.sum(ind):
    # loop through cvs
    filename = test_sess.create_filename(foldername, 1, sub)

    x_train, x_test, x_valid, p_train, p_test, p_valid = prd.train_data_split(raw,params,sub,test_sess.sub_type,dt=test_sess.dt,train_grp=test_sess.train_grp)
    filename = 'subclass/models/TR' + str(sub) + '_tdar'
    with open(filename + '.p', 'rb') as f:
        mlp_w, _, cnn_w, w_sae, c_sae, _, _, w_cnn, c_cnn, w, c, w_noise, c_noise, emg_scale, scaler, mu_class, C = pickle.load(f)
    # Load saved data
    # with open(filename + '.p', 'rb') as f:
    #     scaler, svae_w, svae_enc_w, svae_dec_w, svae_clf_w, sae_w, sae_enc_w, sae_clf_w, cnn_w, cnn_enc_w, cnn_clf_w, vcnn_w, vcnn_enc_w, vcnn_clf_w, ecnn_w, ecnn_enc_w, ecnn_clf_w, w_svae, c_svae, w_sae, c_sae, w_cnn, c_cnn, w_vcnn, c_vcnn, w_ecnn, c_ecnn, w, c, w_noise, c_noise, mu, C, qda, qda_noise,emg_scale = pickle.load(f)   

    # Add noise to training data
    y_shape = np.max(p_train[:,4])

    sae = MLP(n_class=y_shape)
    cnn = CNN(n_class=y_shape)
    
    # Build models and set weights
    # sae, sae_enc, sae_clf = dl.build_sae(test_sess.latent_dim, y_shape, input_type=test_sess.feat_type, sparse=test_sess.sparsity,lr=test_sess.lr)
    # cnn, cnn_enc, cnn_clf = dl.build_cnn(test_sess.latent_dim, y_shape, input_type=test_sess.feat_type, sparse=test_sess.sparsity,lr=test_sess.lr)

    # sae.set_weights(sae_w)
    # sae_enc.set_weights(sae_enc_w)
    # sae_clf.set_weights(sae_clf_w)

    # cnn.set_weights(cnn_w)
    # cnn_enc.set_weights(cnn_enc_w)
    # cnn_clf.set_weights(cnn_clf_w)

    # set test on validation data for cv mode
    x_test = x_test*emg_scale

    # loop through test levels
    noisefolder = test_sess.create_foldername(ftype='testnoise')
    noisefolder = 'testdata/' + noisefolder
    noisefile = test_sess.create_filename(noisefolder,1, sub, ftype='testnoise', test_scale=1)
    if os.path.isfile(noisefile + '.p'):
        print('loading data')
        with open(noisefile + '.p','rb') as f:
            x_test_vae, x_test_clean_vae, x_test_lda, y_test_clean = pickle.load(f) 
        test_load = True
    x_temp = np.transpose(x_test_lda.reshape((x_test_lda.shape[0],num_feats,-1)),(0,2,1))[...,np.newaxis]
    x_test_vae = scaler.transform(x_temp.reshape(x_temp.shape[0]*x_temp.shape[1],-1)).reshape(x_temp.shape)

    # Reshape for nonconvolutional SAE
    x_test_dlsae = x_test_vae.reshape(x_test_vae.shape[0],-1)

    sae(x_test_dlsae[:1,...].astype('float32'))
    cnn(x_test_vae[:1,...].astype('float32'))
    sae.set_weights(mlp_w)
    cnn.set_weights(cnn_w)
    sae_enc = sae.enc
    cnn_enc = cnn.enc
    ## for timing
    %timeit predict(sae_enc(x_test_dlsae[[0],...].astype('float32')).numpy(),w_sae,c_sae)

    %timeit predict(cnn_enc(x_test_vae[[0],...].astype('float32')).numpy(),w_cnn,c_cnn)

    # lda_start = time.time()
    # temp_out = predict(x_test_lda[[0],...], w, c)
    # lda_temp = timeit.timeit(lambda: predict(x_test_lda[[0],...], w, c),number= 100)
    %timeit predict(x_test_lda[[0],...], w, c)

    %timeit predict(x_test_lda[[0],...], w_noise, c_noise)

    temp_x = x_test_lda[[0],...]


In [26]:
test = np.nanmean(ave_noise2,axis=0)

In [30]:
np.nanstd(np.nanmean(ave_clean2,axis=1),axis=0)/7

c:\ProgramData\Anaconda3\envs\latent_env\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


array([0.01875627, 0.01520981, 0.01875627, 0.01875627, 0.01981864,
       0.01766341, 0.01465204, 0.01627478, 0.01627478, 0.01627478,
       0.01422465, 0.01221415, 0.01422465, 0.01422465, 0.01422465])

In [27]:
test2 = np.nanmean(np.nanmean(ave_clean2,axis=1),axis=0)

c:\ProgramData\Anaconda3\envs\latent_env\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


In [9]:
temp = np.hstack((ave_clean2[:,[0],:],ave_noise2))

In [10]:
np.nanmean(temp[:,:,[7]],axis=0) 

array([[0.77169048],
       [0.73663095],
       [0.68446429],
       [0.62946429],
       [0.55695238]])

In [54]:
np.nanmean(temp[:,:,10],axis=0)

array([0.78102381, 0.41516667, 0.30678571, 0.25839286, 0.23329762])

In [55]:
78.1-72.11

5.989999999999995

In [53]:
72.11-66.9

5.209999999999994

In [43]:
temp1=np.nanmean(temp[:,1:,:],axis=1,keepdims=True)

c:\ProgramData\Anaconda3\envs\latent_env\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


In [49]:
np.nanmean(np.nanmean(temp[:,1:,:] - temp[:,1:,[10]],axis=1),axis=0)

c:\ProgramData\Anaconda3\envs\latent_env\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


array([0.33931548, 0.29085119, 0.33931548, 0.33931548, 0.00893155,
       0.03090774, 0.2904881 , 0.34846726, 0.34846726, 0.34846726,
       0.        , 0.18010417, 0.        , 0.        , 0.38867823])

In [51]:
np.nanmean(np.nanmean(temp,axis=0)

In [ ]:
np.nanmean(np.nanmean(temp[:,1:,:] - temp[:,1:,[10]],axis=1),axis=0)

In [28]:
test-test2

array([[-0.03714286, -0.05054762, -0.03714286, -0.03714286, -0.32279762,
        -0.31885714, -0.05210714, -0.03505952, -0.03505952, -0.03505952,
        -0.36585714, -0.04608333, -0.36585714, -0.36585714, -0.02221117],
       [-0.08833333, -0.08888095, -0.08833333, -0.08833333, -0.42390476,
        -0.43736905, -0.08991667, -0.08722619, -0.08722619, -0.08722619,
        -0.4742381 , -0.09439286, -0.4742381 , -0.4742381 , -0.05341536],
       [-0.14778571, -0.14395238, -0.14778571, -0.14778571, -0.46521429,
        -0.50455952, -0.14715476, -0.14222619, -0.14222619, -0.14222619,
        -0.52263095, -0.14629762, -0.52263095, -0.52263095, -0.1059881 ],
       [-0.22321429, -0.20952381, -0.22321429, -0.22321429, -0.48047619,
        -0.53132143, -0.21970238, -0.2147381 , -0.2147381 , -0.2147381 ,
        -0.54772619, -0.20130952, -0.54772619, -0.54772619, -0.17412484]])

In [8]:
test1=np.nanmean(ave_noise2 - np.nanmean(ave_clean2[:,[0],:]),axis=0)

In [9]:
test1

array([[-9.23412698e-03, -6.27460317e-02,  1.59920635e-03,
        -6.62698413e-03, -3.14948413e-01, -2.64103175e-01,
        -6.96031746e-02,  1.50753968e-02,  8.37301587e-04,
        -2.19436508e-01, -3.10210317e-01, -1.71067460e-01,
        -4.34186508e-01, -1.22769841e-01,  3.12213697e-02],
       [-6.69246032e-02, -1.34281746e-01, -5.18174603e-02,
        -5.93293651e-02, -4.16055556e-01, -3.82615079e-01,
        -1.34341270e-01, -3.92698413e-02, -5.21031746e-02,
        -2.51388889e-01, -4.22757937e-01, -2.20460317e-01,
        -5.04757937e-01, -1.72103175e-01,  1.71778011e-05],
       [-1.22686508e-01, -2.09269841e-01, -1.17829365e-01,
        -1.32329365e-01, -4.57365079e-01, -4.49805556e-01,
        -2.03115079e-01, -1.03043651e-01, -1.28496032e-01,
        -2.88365079e-01, -4.65067460e-01, -2.73043651e-01,
        -5.30496032e-01, -2.14031746e-01, -5.25555556e-02],
       [-2.05317460e-01, -2.83222222e-01, -2.05031746e-01,
        -2.12496032e-01, -4.72626984e-01, -4.76567460

In [ ]:
%%timeit
mask = np.ones(6,dtype=bool) 
for i in range(1):
    mask[0] = 0
maskmu = np.tile(mask,4)
test_data = temp_x[:,maskmu]
C_temp = C[maskmu,:]
C_in = C_temp[:,maskmu]
w_temp, c_temp = train_lda(test_data,1,mu_bool = True, mu_class = mu[:,maskmu], C = C_in)

In [ ]:
sae_diff = acc[:,7] - acc[:,14]
sae_diff_tr = acc_tr[:,7] - acc_tr[:,14]

In [ ]:
acc_tr = cp.deepcopy(acc)
std_tr = cp.deepcopy(std)

In [ ]:
plot_utils.plot_summary(ave_clean,ave_mix)

In [ ]:
# plot accuracy vs. position
plot_utils.plot_pos_results(ave_pos_noise)

In [24]:
import statsmodels.formula.api as smf

df = stat.create_dataframe(acc_clean2,acc_noise2)
df = stat.get_mods(df)
all_md = stat.run_fit(df)

                     Mixed Linear Model Regression Results
Model:                    MixedLM         Dependent Variable:         acc      
No. Observations:         210             Method:                     REML     
No. Groups:               6               Scale:                      65.1511  
Min. group size:          35              Log-Likelihood:             -722.2117
Max. group size:          35              Converged:                  Yes      
Mean group size:          35.0                                                 
-------------------------------------------------------------------------------
                                    Coef.  Std.Err.   z    P>|z|  [0.025 0.975]
-------------------------------------------------------------------------------
Intercept                           35.062    3.960  8.854 0.000  27.301 42.823
C(mod, Treatment(10))[T.4.0]         3.174    3.610  0.879 0.379  -3.901 10.249
C(mod, Treatment(10))[T.5.0]        -2.421    3.610 -0.671 0.

In [25]:
for i in range(5):
    # print(all_md[str(i) + '.0'].pvalues)
    print(all_md[str(i) + '.0'].pvalues*5)

Intercept                        4.518991e-05
C(mod, Treatment(10))[T.4.0]     2.542982e-02
C(mod, Treatment(10))[T.5.0]     4.676356e+00
C(mod, Treatment(10))[T.6.0]     1.158289e-03
C(mod, Treatment(10))[T.7.0]     2.831112e+00
C(mod, Treatment(10))[T.11.0]    2.016412e-26
C(mod, Treatment(10))[T.14.0]    5.000000e+00
Group Var                        7.326145e-01
dtype: float64
Intercept                        8.656530e-42
C(mod, Treatment(10))[T.4.0]     4.740756e+00
C(mod, Treatment(10))[T.5.0]     1.065562e+00
C(mod, Treatment(10))[T.6.0]     3.063817e-10
C(mod, Treatment(10))[T.7.0]     6.012243e-16
C(mod, Treatment(10))[T.11.0]    1.005328e-03
C(mod, Treatment(10))[T.14.0]    4.206704e-18
Group Var                        9.156474e-01
dtype: float64
Intercept                        5.175089e-77
C(mod, Treatment(10))[T.4.0]     4.432325e+00
C(mod, Treatment(10))[T.5.0]     1.254938e+00
C(mod, Treatment(10))[T.6.0]     9.086079e-22
C(mod, Treatment(10))[T.7.0]     3.603475e-29
C(mo